# Lab 2, Question 6

(6) Exercise: using an SVM model and the Landcare NZ 2024 landcover database, produce a landcover map of Great Barrier (Aotea) Island for 2025 (based off the Austral summer of 24/25).

Your map should be presented at a publication quality level with all the usual map components (scale, legend, north arrow, data attribution).

You will need to provide performance statistics of the model within your figure.

*   Here you can access the landcover database: https://lris.scinfo.org.nz/layer/104400-lcdb-v50-land-cover-database-version-50-mainland-new-zealand/. You will need to explore for yourself how to extract this data and then upload it to colab, then how to plug it into the SVM algorithim. I have provided some starter code below.

An intial workflow to get the data into the state you need it in to then use it as training data might look like:
- Download the ZIP manually from their browser, having set your area of interest and used the 'Export' tool top right.
- Upload it to Colab.
- Unzip it and load with GeoPandas.
(25 pts)


_______________________________________________________________________________


### Setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID) #<- Remember to change this to your own project's name!

In [ ]:
import tempfile
import urllib.request

from IPython.display import Image

import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sklearn.metrics import confusion_matrix, classification_report

# A. Read Data

### A.1. Load LCDB v5.0 - Land Cover Database version 5.0, Mainland, New Zealand for 2024

In [ ]:
# # Code to get you started
# import zipfile
# import geopandas as gpd

# # Upload the ZIP manually using the Colab UI
# from google.colab import files
# uploaded = files.upload()  # <- Expects a ZIP

# Unzip
# with zipfile.ZipFile("LCDB_v5.zip", 'r') as zip_ref: #<- Check file names
#     zip_ref.extractall("lcdb")

# Read shapefile
gdf = gpd.read_file("../dataset/lris-lcdb-v50-land-cover-great-barrier-SHP/lcdb-v50-land-cover-database-version-50-mainland-new-zealand.shp")

In [ ]:
df_2018 = gdf[['Name_2018', 'Class_2018', 'geometry', 'LCDB_UID']]

# original class of 2018
classes_2018 = [0, 1, 2, 5, 6, 10, 12, 14, 16, 15, 20, 21, 22, 30, 33, 40, 41, 43, 44, 45, 46, 47, 50, 51, 52, 54, 55, 56, 58, 80, 81, 70, 64, 68, 69, 71]
class_mapped_2018 = list(range(1, 37)) # Creates a list [1, 2, 3, ..., 36], to map from original

class_property = 'class_mapped_2018'
df_2018[class_property] = df_2018['Class_2018'].map(dict(zip(classes_2018, class_mapped_2018)))
df_2018

In [ ]:
seed = 42  # Random seed for reproducibility

# # Coordinates for Mount Hobson
# point = ee.Geometry.Point([175.3785, -36.1830])
aoi = ee.Geometry.Rectangle([175.27, -36.01, 175.57, -36.36])

# bands = ['B2', 'B3', 'B4', 'B8'] # Sentinel-2 bands: Blue, Green, Red, NIR
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']

lcdb = geemap.geopandas_to_ee(df_2018[[class_property, 'geometry']])

# rasterizes the data
lcdb_image = lcdb.reduceToImage(
    properties=[class_property],
    reducer=ee.Reducer.first()
).rename(class_property).clip(aoi)

def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    # Bits 3 and 4 are cloud and cloud shadow respectively
    # .eq(0) returns True for pixels where the bit is not set
    #  (i.e., not cloudy AND not shadow).
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0).And(
                 qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(cloud_mask)

l8 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterBounds(aoi)
           .filter(ee.Filter.lt('CLOUD_COVER', 10)))

l8_2018 = (l8
           .filterDate('2018-01-01', '2018-12-31')
           .map(mask_clouds)
           .median()
           .clip(aoi))

# `lcdb` only covers land. Need to mark unlabled areas in `l8_2018`.
l8_2018_lc = l8_2018.updateMask(lcdb_image)
training_image = l8_2018_lc.addBands(lcdb_image)

# Combine the bands of interest with the land cover data
# The `class_property` band will be used as our label
training_image = l8_2018_lc.addBands(lcdb_image)

### Visualize

In [ ]:
# Add the layers to the map.
# point = ee.Geometry.Point([175.3785, -36.1830])
viz_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 5000,
    'max': 30000,
    # 'gamma': 0.8
}

Map = geemap.Map(center=[-36.1830, 175.3785], zoom=11)
Map.addLayer(aoi, {}, 'AOI')

Map.add_layer(
    l8_2018, viz_params, 'L8 2018 Median RGB'
)
Map.add_layer(
    l8_2018_lc, viz_params, 'L8 2018 (Land Cover)'
)
Map.add_layer(
    lcdb_image.randomVisualizer(), {}, 'LCDB v5.0 (2018)'
)
Map

### Training Data 

In [ ]:
# # Sample the image to create training data
# # This will create a FeatureCollection where each feature has the pixel values 
# # for the bands and the corresponding land cover class.
training_data = training_image.select(bands + [class_property]).stratifiedSample(
    classBand=class_property,
    region=aoi,
    scale=30,  # The spatial resolution to sample at
    numPoints=150,  # The number of data points for each class
    seed=seed,  # Random seed for reproducibility
    geometries=True,  # Include the geometry of each sampled pixel
)

# training_data = training_image.select(bands + [class_property]).sample(
#     region=aoi,
#     scale=30,
#     numPixels=5000,
#     seed=seed,
#     geometries=True)

In [ ]:
first_10 = training_data.limit(10).getInfo()

print('\nFirst 10 training samples:')
for i, feature in enumerate(first_10['features']):
    props = feature['properties']
    print(f"Sample {i+1}: Class={props[class_property]}, SR_B2={props['SR_B2']}, SR_B3={props['SR_B3']}, SR_B4={props['SR_B4']}, SR_B7={props['SR_B7']}")

# Count number of samples per class (server-side)
class_counts = training_data.reduceColumns(
    reducer=ee.Reducer.frequencyHistogram(),
    selectors=[class_property])

print('\nClass distribution in sample:')
print(class_counts.getInfo())

In [ ]:
# Add random column
training_data = training_data.randomColumn('random')

# Split
train_set = training_data.filter(ee.Filter.lt('random', 0.7))
valid_set = training_data.filter(ee.Filter.And(ee.Filter.gte('random', 0.7), ee.Filter.lt('random', 0.9)))
test_set = training_data.filter(ee.Filter.gte('random', 0.9))

In [ ]:
classifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train(
    features=train_set,
    classProperty=class_property,
    inputProperties=bands)

In [ ]:
display('Results of trained classifier', classifier.explain())

In [ ]:
# Get a confusion matrix and overall accuracy for the training sample.
train_accuracy = classifier.confusionMatrix()
display('Training overall accuracy', train_accuracy.accuracy())

In [ ]:
# Classify validation and test sets
val_classified = valid_set.classify(classifier)
test_classified = test_set.classify(classifier)

# Function to export data for confusion matrix
def fc_to_lists(fc, classProp, predProp):
    values = fc.aggregate_array(classProp).getInfo()
    preds = fc.aggregate_array(predProp).getInfo()
    return values, preds

# Get predicted vs actual from validation set
y_true, y_pred = fc_to_lists(val_classified, class_property, 'classification')


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=list(range(1, 37)))
report = classification_report(y_true, y_pred, labels=list(range(1, 37)), target_names=[str(l) for l in classes_2018])

# Pretty-print
print("Confusion Matrix:")
print(pd.DataFrame(cm, index=[f"Actual {l}" for l in classes_2018],
                       columns=[f"Pred {l}" for l in classes_2018]))
print("\nClassification Report:")
print(report)

In [ ]:

# Evaluate test performance
test_matrix = test_classified.errorMatrix(class_property, 'classification')
# print("Confusion Matrix:")
# print(pd.DataFrame(test_matrix.getInfo(), index=[f"Actual {l}" for l in classes_2018],
#                        columns=[f"Pred {l}" for l in classes_2018]))
# print("\nClassification Report:")
# print(report)

# print('Error matrix:')
# print(test_matrix.getInfo())

# Compute accuracy metrics from the error matrix.
print('Overall accuracy:', test_matrix.accuracy().getInfo())
# print('Consumer\'s accuracy:')
# print(test_matrix.consumersAccuracy().getInfo())
# print('Producer\'s accuracy:')
# print(test_matrix.producersAccuracy().getInfo())
print('Kappa:', test_matrix.kappa().getInfo())

In [ ]:
# Austral summer of 24/25
l8_2025 = (l8
           .filterDate('2024-09-01', '2025-07-31')
           .map(mask_clouds)
           .median()
           .clip(aoi))

l8_2025_classified = l8_2025.updateMask(lcdb_image).classify(classifier)

l8_2018_classified = l8_2018_lc.classify(classifier)

In [ ]:
# Classified image viz, fresh set up again to make sure we have exactly what we want displayed.

classified_map = geemap.Map()
classified_map.centerObject(aoi, 10)
classified_map.addLayer(l8_2025, viz_params, '2025 RGB')
classified_map.addLayer(l8_2025_classified.randomVisualizer(), {}, '2025 Classified')
classified_map.addLayer(l8_2018_lc, viz_params, '2018 RGB')
classified_map.addLayer(l8_2018_classified.randomVisualizer(), {}, '2018 Classified')
classified_map.addLayer(lcdb_image.randomVisualizer(), {} ,'LCDB v5.0')

classified_map